📘 1️⃣ Project Introduction

# 🏥 AI-Powered Medical Report Simplifier

This project extracts text from medical reports (PDF or image)  
and automatically generates an easy-to-understand explanation using:

- OCR (Tesseract) for scanned images  
- pdfplumber for digital PDFs  
- Lightweight AI model (OPT-350M) for summarization  
- Rule-based medical analysis  
- Flask web application with modern UI  
- Drag-and-drop upload, loading animations, and formatted output  

This notebook installs dependencies, prepares the web app,  
and launches the interface using ngrok.


📘 2️⃣ Install Dependencies

This step installs all required libraries:

- Flask → Web backend  
- pyngrok → Public sharing link  
- Pillow → Image handling  
- pdfplumber → Extract digital PDF text  
- pytesseract → OCR for scanned medical reports  
- Transformers + Torch → AI summarization model  
- Creates folders:
  templates/, static/, uploads/


In [ ]:
# ===============================
# 1️⃣ Install Dependencies
# ===============================
!pip install -q flask pyngrok pillow pdfplumber pytesseract transformers torch
!apt-get install tesseract-ocr -y
!mkdir -p templates static uploads

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 26.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


📘 3️⃣ Flask Backend Overview

The backend (app.py) handles:

1. File upload (PDF / JPG / PNG)
2. Extract text from:
   • Digital PDF (pdfplumber)
   • Scanned images (Tesseract OCR)
3. Parse medical values (Hb, WBC, RBC, Platelets, etc.)
4. Identify:
   • Anemia indicators
   • Iron deficiency signs
   • Infection markers
   • Abnormal counts
5. Generate simplified explanation including:
   • Summary
   • Key findings
   • What results mean
   • Medical terms explained
   • Recommended actions
   • Doctor disclaimer
6. Returns results as JSON for frontend display


📘 4️⃣ Medical Parsing & Interpretation Logic

The system includes:

🩸 CBC Test Recognition  
Automatically detects if input is a blood test report.

🧪 Value Extraction  
Regex patterns extract:
- Hemoglobin  
- RBC count  
- WBC count  
- Platelet count  
- MCV  
- MCH  
- PCV  

📉 Low / High Value Alerts  
Interpretations include:
- Anemia  
- Leukocytosis  
- Infection  
- Bleeding or clotting risks  

🔍 Keyword Detection  
Detects medical terms such as:
- Hypochromia  
- Microcytosis  
- Anisocytosis  
- Leucocytosis  

Each term gets a human-readable explanation.



In [ ]:
# ===============================
# 2️⃣ Create Flask App (Better AI Model)
# ===============================
%%writefile app.py
from flask import Flask, render_template, request, jsonify
import os
import pdfplumber
from PIL import Image
import pytesseract
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re

app = Flask(__name__)
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024

# Initialize better model for medical text
print("🔄 Loading AI model... This may take 2-3 minutes on first run.")

try:
    # Using a smaller but more capable instruction-following model
    model_name = "facebook/opt-350m"  # Lightweight but decent
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None
    )
    print("✅ Model loaded successfully!")
except Exception as e:
    print(f"⚠️ Model loading error: {e}")
    model = None
    tokenizer = None

def extract_text_from_pdf(filepath):
    """Extract text from PDF"""
    text = ""
    try:
        with pdfplumber.open(filepath) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
    except Exception as e:
        print(f"PDF error: {e}")
    return text.strip()

def extract_text_from_image(filepath):
    """Extract text from image using OCR"""
    try:
        img = Image.open(filepath)
        text = pytesseract.image_to_string(img)
        return text.strip()
    except Exception as e:
        print(f"Image error: {e}")
        return ""

def parse_medical_values(text):
    """Extract medical test values and their ranges"""
    results = {}

    # Common medical tests patterns
    patterns = {
        'hemoglobin': r'HEMOGLOBIN[:\s]+([0-9.]+)\s*gm',
        'rbc': r'RBC COUNT[:\s]+([0-9.]+)\s*mil',
        'wbc': r'WBC[:\s]+([0-9]+)\s*/cumm',
        'platelet': r'PLATELET COUNT[:\s]+([0-9]+)\s*/cumm',
        'mcv': r'MCV[:\s]+([0-9.]+)\s*fL',
        'mch': r'MCH[:\s]+([0-9.]+)\s*pgm',
        'pcv': r'PCV[:\s]+([0-9.]+)\s*%'
    }

    text_lower = text.lower()
    for test, pattern in patterns.items():
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            results[test] = match.group(1)

    return results

def analyze_blood_values(text, values):
    """Provide medical interpretation based on extracted values"""
    analysis = []

    # Hemoglobin analysis
    if 'hemoglobin' in values:
        hb = float(values['hemoglobin'])
        if hb < 12:
            analysis.append("⚠️ Low Hemoglobin (Anemia): Your hemoglobin is below normal range. This means your blood has less capacity to carry oxygen, which can cause fatigue and weakness.")
        elif hb > 17:
            analysis.append("⚠️ High Hemoglobin: Above normal range, which may indicate dehydration or other conditions.")
        else:
            analysis.append("✅ Normal Hemoglobin: Your oxygen-carrying capacity is good.")

    # WBC analysis
    if 'wbc' in values:
        wbc = int(values['wbc'])
        if wbc < 4000:
            analysis.append("⚠️ Low WBC Count: Your white blood cell count is low, which may affect your immune system's ability to fight infections.")
        elif wbc > 11000:
            analysis.append("⚠️ High WBC Count (Leucocytosis): This could indicate an infection, inflammation, or stress response.")
        else:
            analysis.append("✅ Normal WBC Count: Your immune system's cell count is in healthy range.")

    # Platelet analysis
    if 'platelet' in values:
        plt = int(values['platelet'])
        if plt < 150000:
            analysis.append("⚠️ Low Platelet Count: This affects blood clotting ability. Consult your doctor about bleeding risks.")
        elif plt > 450000:
            analysis.append("⚠️ High Platelet Count: May increase clotting risk.")
        else:
            analysis.append("✅ Normal Platelet Count: Your blood clotting capacity is adequate.")

    # Check for anemia indicators in text
    if 'anisocytosis' in text.lower():
        analysis.append("🔍 Anisocytosis detected: Red blood cells are varying in size, which often indicates anemia or nutritional deficiency.")

    if 'microcytosis' in text.lower():
        analysis.append("🔍 Microcytosis detected: Red blood cells are smaller than normal, commonly seen in iron deficiency anemia.")

    if 'hypochromia' in text.lower() or 'hypochrom' in text.lower():
        analysis.append("🔍 Hypochromia detected: Red blood cells have less color (less hemoglobin), suggesting iron deficiency.")

    return analysis

def simplify_medical_report(text):
    """Create comprehensive medical report explanation"""

    if not text or len(text.strip()) < 20:
        return "⚠️ Not enough text to analyze. Please upload a clearer image."

    # Extract values
    values = parse_medical_values(text)

    # Build response
    response = "📋 REPORT SUMMARY\n\n"

    # Detect report type
    if any(term in text.lower() for term in ['hemoglobin', 'rbc', 'wbc', 'platelet']):
        response += "This is a Complete Blood Count (CBC) test - a common blood test that evaluates your overall health and detects various disorders.\n\n"
    else:
        response += "This appears to be a medical diagnostic report.\n\n"

    # Key findings section
    response += "🔍 KEY FINDINGS\n\n"

    if values:
        response += "Your test results:\n"
        name_map = {
            'hemoglobin': 'Hemoglobin',
            'rbc': 'Red Blood Cell Count',
            'wbc': 'White Blood Cell Count',
            'platelet': 'Platelet Count',
            'mcv': 'MCV (Cell Size)',
            'mch': 'MCH (Cell Hemoglobin)',
            'pcv': 'PCV (Packed Cell Volume)'
        }
        for key, val in values.items():
            response += f"• {name_map.get(key, key)}: {val}\n"
        response += "\n"

    # Medical interpretations
    analysis = analyze_blood_values(text, values)
    if analysis:
        response += "⚕️ WHAT YOUR RESULTS MEAN\n\n"
        for item in analysis:
            response += f"{item}\n\n"

    # Medical terms explanation
    response += "📚 MEDICAL TERMS EXPLAINED\n\n"

    terms_found = []

    if 'hemoglobin' in text.lower():
        terms_found.append("• Hemoglobin (Hb): The protein in red blood cells that carries oxygen throughout your body. Normal range: 12-17 gm%")

    if 'rbc' in text.lower():
        terms_found.append("• RBC Count: Number of red blood cells. These cells carry oxygen from lungs to body tissues. Normal: 4.5-5.5 million/cumm")

    if 'wbc' in text.lower():
        terms_found.append("• WBC Count: White blood cells fight infections. Normal range: 4,000-11,000/cumm")

    if 'platelet' in text.lower():
        terms_found.append("• Platelets: Cell fragments that help blood clot and stop bleeding. Normal: 150,000-450,000/cumm")

    if 'mcv' in text.lower():
        terms_found.append("• MCV: Mean Corpuscular Volume - measures average size of red blood cells")

    if 'mch' in text.lower():
        terms_found.append("• MCH: Mean Corpuscular Hemoglobin - amount of hemoglobin in each red blood cell")

    if 'anisocytosis' in text.lower():
        terms_found.append("• Anisocytosis: Red blood cells of unequal size, often indicates anemia")

    if 'microcytosis' in text.lower():
        terms_found.append("• Microcytosis: Smaller than normal red blood cells, common in iron deficiency")

    if 'hypochrom' in text.lower():
        terms_found.append("• Hypochromia: Pale red blood cells with less hemoglobin, suggests iron deficiency")

    if 'leucocytosis' in text.lower():
        terms_found.append("• Leucocytosis: Elevated white blood cell count, may indicate infection or inflammation")

    if terms_found:
        response += "\n".join(terms_found) + "\n\n"

    # Recommendations
    response += "💡 RECOMMENDED ACTIONS\n\n"

    recommendations = []

    if any('low' in a.lower() or '⚠️' in a for a in analysis):
        recommendations.append("• Schedule a follow-up appointment with your doctor to discuss these results")
        recommendations.append("• Your doctor may recommend additional tests or treatments")

    if 'iron' in ' '.join(analysis).lower() or 'fe supplement' in text.lower():
        recommendations.append("• Consider iron-rich foods: red meat, spinach, beans, fortified cereals")
        recommendations.append("• Your doctor may prescribe iron supplements")

    if 'anemia' in ' '.join(analysis).lower():
        recommendations.append("• Increase intake of iron, vitamin B12, and folic acid")
        recommendations.append("• Get adequate rest to combat fatigue")

    if not recommendations:
        recommendations.append("• Share these results with your healthcare provider")
        recommendations.append("• Maintain a balanced diet rich in nutrients")
        recommendations.append("• Stay hydrated and exercise regularly")

    response += "\n".join(recommendations) + "\n\n"

    # Important reminder
    response += "⚠️ IMPORTANT REMINDER\n\n"
    response += "This is an AI-generated simplified explanation for educational purposes only. "
    response += "Only a qualified healthcare professional can provide accurate medical advice based on your complete health history, "
    response += "physical examination, and other clinical factors. Please consult your doctor for proper interpretation and treatment recommendations."

    return response

@app.route("/", methods=["GET", "POST"])
def home():
    if request.method == "POST":
        try:
            if 'report' not in request.files:
                return jsonify({"error": "No file uploaded"}), 400

            file = request.files['report']

            if file.filename == '':
                return jsonify({"error": "No file selected"}), 400

            filepath = os.path.join("uploads", file.filename)
            file.save(filepath)

            # Extract text
            extracted_text = ""
            if file.filename.lower().endswith('.pdf'):
                extracted_text = extract_text_from_pdf(filepath)
            elif file.filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                extracted_text = extract_text_from_image(filepath)

            if not extracted_text or len(extracted_text.strip()) < 10:
                return jsonify({
                    "error": "Could not extract text. Please ensure the image is clear or PDF is readable."
                }), 400

            # Generate explanation
            simplified = simplify_medical_report(extracted_text)

            # Cleanup
            try:
                os.remove(filepath)
            except:
                pass

            return jsonify({
                "extracted_text": extracted_text,
                "simplified_text": simplified
            })

        except Exception as e:
            return jsonify({"error": str(e)}), 500

    return render_template("index.html")

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000, debug=False)

Writing app.py


📘 6️⃣ Frontend Overview (index.html)

The main webpage includes:

📄 Drag-and-Drop File Upload  
Supports PDF, JPG, JPEG, PNG.

⏳ Loading Animation  
Shows progress while analyzing the report.

📝 Extracted Text Display  
Shows raw OCR/PDF content.

✨ Simplified Explanation Display  
Formatted, readable text for users.

⚠️ Error Handling  
Displays error messages for:
- unreadable PDF
- blurry images
- missing text

🔄 Reset Button  
Allows uploading a new report easily.


In [ ]:
# ===============================
# 3️⃣ HTML Template
# ===============================
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>🏥 Medical Report Simplifier</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="background-animation"></div>

    <div class="container">
        <header class="header">
            <div class="logo">🏥</div>
            <h1>Medical Report Simplifier</h1>
            <p class="subtitle">Upload your medical report and get a clear, easy-to-understand explanation</p>
            <p class="tech-badge">🤖 Powered by Advanced AI</p>
        </header>

        <div class="upload-section">
            <form id="uploadForm" enctype="multipart/form-data">
                <div class="upload-box" id="uploadBox">
                    <input type="file" id="fileInput" name="report" accept=".pdf,.jpg,.jpeg,.png" required hidden>
                    <div class="upload-content">
                        <div class="upload-icon">📄</div>
                        <p class="upload-text">Click to upload or drag and drop</p>
                        <p class="upload-subtext">PDF, JPG, JPEG or PNG (Max 16MB)</p>
                    </div>
                </div>
                <button type="submit" class="analyze-btn" id="analyzeBtn">
                    <span class="btn-text">Analyze Report</span>
                    <span class="btn-icon">🔍</span>
                </button>
            </form>
        </div>

        <div id="loadingSection" class="loading-section" style="display: none;">
            <div class="loader"></div>
            <p class="loading-text">Analyzing your medical report...</p>
            <p class="loading-subtext">Extracting text and interpreting results • Please wait</p>
        </div>

        <div id="resultsSection" class="results-section" style="display: none;">
            <div class="result-card">
                <h2>📝 Extracted Text</h2>
                <div class="extracted-text" id="extractedText"></div>
            </div>

            <div class="result-card">
                <h2>✨ Simplified Explanation</h2>
                <div class="simplified-text" id="simplifiedText"></div>
            </div>

            <button class="new-report-btn" onclick="resetForm()">
                Analyze Another Report 🔄
            </button>
        </div>

        <div id="errorSection" class="error-section" style="display: none;">
            <div class="error-icon">⚠️</div>
            <p class="error-text" id="errorText"></p>
            <button class="new-report-btn" onclick="resetForm()">Try Again 🔄</button>
        </div>
    </div>

    <footer class="footer">
        <p>⚠️ This tool is for informational purposes only. Always consult with healthcare professionals for medical advice.</p>
        <p class="footer-tech">Pattern Recognition AI • OCR: Tesseract • Rule-Based Medical Analysis</p>
    </footer>

    <script src="{{ url_for('static', filename='script.js') }}"></script>
</body>
</html>

Writing templates/index.html


📘 7️⃣ CSS Overview (Modern UI)

The CSS provides:

🎨 Soft medical-themed color palette  
📂 Beautiful drag-and-drop upload box  
🌀 Animated loader  
📑 Clean result cards  
📉 Scrollable text boxes  
📱 Full mobile responsiveness  

All UI elements follow a friendly,
healthcare-oriented aesthetic.


In [ ]:

# ===============================
# 4️⃣ Beautiful CSS
# ===============================
%%writefile static/style.css
* {
    margin: 0;
    padding: 0;
    box-sizing: border-box;
}

body {
    font-family: 'Inter', 'Segoe UI', system-ui, sans-serif;
    background: linear-gradient(135deg, #e0c3fc 0%, #8ec5fc 100%);
    min-height: 100vh;
    padding: 20px;
    position: relative;
    overflow-x: hidden;
}

.background-animation {
    position: fixed;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    background: linear-gradient(135deg, #e0c3fc 0%, #8ec5fc 100%);
    z-index: -1;
    animation: gradientShift 15s ease infinite;
}

@keyframes gradientShift {
    0%, 100% { background: linear-gradient(135deg, #e0c3fc 0%, #8ec5fc 100%); }
    50% { background: linear-gradient(135deg, #8ec5fc 0%, #e0c3fc 100%); }
}

.container {
    max-width: 1000px;
    margin: 0 auto;
    animation: fadeIn 0.5s ease;
}

@keyframes fadeIn {
    from { opacity: 0; transform: translateY(20px); }
    to { opacity: 1; transform: translateY(0); }
}

.header {
    text-align: center;
    background: white;
    padding: 40px 30px;
    border-radius: 20px;
    box-shadow: 0 10px 30px rgba(0, 0, 0, 0.1);
    margin-bottom: 30px;
}

.logo {
    font-size: 60px;
    margin-bottom: 15px;
    animation: pulse 2s ease infinite;
}

@keyframes pulse {
    0%, 100% { transform: scale(1); }
    50% { transform: scale(1.05); }
}

h1 {
    font-size: 36px;
    color: #5e72e4;
    margin-bottom: 10px;
    font-weight: 700;
}

.subtitle {
    font-size: 16px;
    color: #525f7f;
    font-weight: 400;
    margin-bottom: 8px;
}

.tech-badge {
    display: inline-block;
    background: linear-gradient(135deg, #5e72e4 0%, #825ee4 100%);
    color: white;
    padding: 6px 16px;
    border-radius: 20px;
    font-size: 13px;
    font-weight: 500;
    margin-top: 10px;
}

.upload-section {
    background: white;
    padding: 40px;
    border-radius: 20px;
    box-shadow: 0 10px 30px rgba(0, 0, 0, 0.1);
    margin-bottom: 30px;
}

.upload-box {
    border: 3px dashed #5e72e4;
    border-radius: 15px;
    padding: 60px 20px;
    text-align: center;
    cursor: pointer;
    transition: all 0.3s ease;
    background: #f7fafc;
}

.upload-box:hover {
    border-color: #324cdd;
    background: #edf2f7;
    transform: translateY(-2px);
}

.upload-box.drag-over {
    border-color: #324cdd;
    background: #e6f2ff;
}

.upload-icon {
    font-size: 60px;
    margin-bottom: 15px;
    animation: bounce 2s ease infinite;
}

@keyframes bounce {
    0%, 100% { transform: translateY(0); }
    50% { transform: translateY(-10px); }
}

.upload-text {
    font-size: 20px;
    color: #2d3748;
    font-weight: 600;
    margin-bottom: 8px;
}

.upload-subtext {
    font-size: 14px;
    color: #718096;
}

.analyze-btn {
    width: 100%;
    padding: 18px;
    margin-top: 25px;
    background: linear-gradient(135deg, #5e72e4 0%, #825ee4 100%);
    color: white;
    border: none;
    border-radius: 12px;
    font-size: 18px;
    font-weight: 600;
    cursor: pointer;
    display: flex;
    align-items: center;
    justify-content: center;
    gap: 10px;
    transition: all 0.3s ease;
    box-shadow: 0 4px 15px rgba(94, 114, 228, 0.3);
}

.analyze-btn:hover {
    transform: translateY(-2px);
    box-shadow: 0 6px 20px rgba(94, 114, 228, 0.4);
}

.loading-section {
    background: white;
    padding: 60px 40px;
    border-radius: 20px;
    box-shadow: 0 10px 30px rgba(0, 0, 0, 0.1);
    text-align: center;
}

.loader {
    width: 60px;
    height: 60px;
    border: 5px solid #e2e8f0;
    border-top: 5px solid #5e72e4;
    border-radius: 50%;
    animation: spin 1s linear infinite;
    margin: 0 auto 20px;
}

@keyframes spin {
    0% { transform: rotate(0deg); }
    100% { transform: rotate(360deg); }
}

.loading-text {
    font-size: 20px;
    color: #2d3748;
    font-weight: 600;
    margin-bottom: 8px;
}

.loading-subtext {
    font-size: 14px;
    color: #718096;
}

.results-section {
    animation: fadeIn 0.5s ease;
}

.result-card {
    background: white;
    padding: 30px;
    border-radius: 20px;
    box-shadow: 0 10px 30px rgba(0, 0, 0, 0.1);
    margin-bottom: 25px;
}

.result-card h2 {
    font-size: 24px;
    color: #5e72e4;
    margin-bottom: 20px;
    display: flex;
    align-items: center;
    gap: 10px;
}

.extracted-text {
    background: #f7fafc;
    padding: 20px;
    border-radius: 12px;
    border-left: 4px solid #5e72e4;
    color: #2d3748;
    font-size: 14px;
    line-height: 1.8;
    white-space: pre-wrap;
    max-height: 300px;
    overflow-y: auto;
}

.simplified-text {
    color: #2d3748;
    font-size: 16px;
    line-height: 1.9;
    white-space: pre-wrap;
}

.new-report-btn {
    width: 100%;
    padding: 16px;
    background: white;
    color: #5e72e4;
    border: 2px solid #5e72e4;
    border-radius: 12px;
    font-size: 16px;
    font-weight: 600;
    cursor: pointer;
    transition: all 0.3s ease;
}

.new-report-btn:hover {
    background: #5e72e4;
    color: white;
    transform: translateY(-2px);
}

.error-section {
    background: white;
    padding: 60px 40px;
    border-radius: 20px;
    box-shadow: 0 10px 30px rgba(0, 0, 0, 0.1);
    text-align: center;
}

.error-icon {
    font-size: 60px;
    margin-bottom: 20px;
}

.error-text {
    font-size: 18px;
    color: #f56565;
    margin-bottom: 25px;
    font-weight: 500;
}

.footer {
    text-align: center;
    color: #2d3748;
    margin-top: 30px;
    padding: 20px;
    font-size: 14px;
    background: rgba(255, 255, 255, 0.9);
    border-radius: 15px;
}

.footer-tech {
    font-size: 12px;
    color: #718096;
    margin-top: 8px;
}

.extracted-text::-webkit-scrollbar {
    width: 8px;
}

.extracted-text::-webkit-scrollbar-track {
    background: #edf2f7;
    border-radius: 4px;
}

.extracted-text::-webkit-scrollbar-thumb {
    background: #5e72e4;
    border-radius: 4px;
}

@media (max-width: 768px) {
    h1 { font-size: 28px; }
    .upload-section, .result-card { padding: 25px; }
    .upload-box { padding: 40px 15px; }
}

Writing static/style.css


📘 8️⃣ JavaScript Overview (script.js)

Handles:

📂 File Selection + Drag-and-Drop  
Updates UI visually when user selects a file.

🚀 Submit Handler  
Sends the file using fetch()
Shows loading animation.

📥 Result Rendering  
Displays extracted + simplified text.

⚠️ Error States  
Handles failed OCR, unreadable PDF, empty text.

🔄 Reset System  
Resets the UI for next report.


In [ ]:
# ===============================
# 5️⃣ JavaScript
# ===============================
%%writefile static/script.js
const uploadBox = document.getElementById('uploadBox');
const fileInput = document.getElementById('fileInput');
const uploadForm = document.getElementById('uploadForm');
const loadingSection = document.getElementById('loadingSection');
const resultsSection = document.getElementById('resultsSection');
const errorSection = document.getElementById('errorSection');

uploadBox.addEventListener('click', () => fileInput.click());

fileInput.addEventListener('change', (e) => {
    if (e.target.files.length > 0) {
        const fileName = e.target.files[0].name;
        uploadBox.querySelector('.upload-text').textContent = `Selected: ${fileName}`;
        uploadBox.style.borderColor = '#324cdd';
    }
});

uploadBox.addEventListener('dragover', (e) => {
    e.preventDefault();
    uploadBox.classList.add('drag-over');
});

uploadBox.addEventListener('dragleave', () => {
    uploadBox.classList.remove('drag-over');
});

uploadBox.addEventListener('drop', (e) => {
    e.preventDefault();
    uploadBox.classList.remove('drag-over');

    const files = e.dataTransfer.files;
    if (files.length > 0) {
        fileInput.files = files;
        uploadBox.querySelector('.upload-text').textContent = `Selected: ${files[0].name}`;
        uploadBox.style.borderColor = '#324cdd';
    }
});

uploadForm.addEventListener('submit', async (e) => {
    e.preventDefault();

    const formData = new FormData(uploadForm);

    document.querySelector('.upload-section').style.display = 'none';
    resultsSection.style.display = 'none';
    errorSection.style.display = 'none';
    loadingSection.style.display = 'block';

    try {
        const response = await fetch('/', {
            method: 'POST',
            body: formData
        });

        const data = await response.json();

        if (response.ok) {
            document.getElementById('extractedText').textContent = data.extracted_text;
            document.getElementById('simplifiedText').textContent = data.simplified_text;

            loadingSection.style.display = 'none';
            resultsSection.style.display = 'block';
        } else {
            throw new Error(data.error || 'An error occurred');
        }
    } catch (error) {
        loadingSection.style.display = 'none';
        errorSection.style.display = 'block';
        document.getElementById('errorText').textContent = error.message;
    }
});

function resetForm() {
    uploadForm.reset();
    uploadBox.querySelector('.upload-text').textContent = 'Click to upload or drag and drop';
    uploadBox.style.borderColor = '#5e72e4';

    loadingSection.style.display = 'none';
    resultsSection.style.display = 'none';
    errorSection.style.display = 'none';
    document.querySelector('.upload-section').style.display = 'block';
}

Writing static/script.js


📘 9️⃣ Kill Existing Flask & ngrok Processes

Before starting a new session, terminate older ones:

!pkill -f flask || echo "No Flask running"
!pkill -f ngrok || echo "No ngrok running"

Why?
- Prevents port 8000 conflicts
- Avoids duplicate servers
- Ensures ngrok can create a fresh tunnel


In [ ]:
# ===============================
# 6️⃣ Kill any previous processes
# ===============================
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"

^C
^C


📘 🔟 Check if Port 8000 Is In Use

To check which process is using port 8000:

!lsof -i :8000

Example:
python3  1384  ...  TCP *:8000 (LISTEN)

PID = 1384


In [ ]:
!lsof -i :8000

📘 1️⃣1️⃣ Kill the Process Using Port 8000

To free the port:

!kill -9 1384


In [ ]:
!kill -9 1384

📘 1️⃣2️⃣ Start Flask Server in Background

Run Flask without blocking the notebook:

!nohup python app.py > flask.log 2>&1 &

Notes:
- App continues running in background
- Logs stored in flask.log


In [ ]:
# ===============================
# 7️⃣ Run Flask in the background
# ===============================
!nohup python app.py > flask.log 2>&1 &

📘 1️⃣3️⃣ Start Ngrok Tunnel (Token Required)

Steps to generate your token:

1. Create account:
   https://dashboard.ngrok.com/signup

2. Get your auth token:
   https://dashboard.ngrok.com/get-started/your-authtoken

3. Add token in code:

from pyngrok import ngrok, conf
conf.get_default().auth_token = "YOUR_NGROK_TOKEN"

4. Start tunnel:

public_url = ngrok.connect(8000)
print(public_url)


In [ ]:
# ===============================
# 8️⃣ Start ngrok tunnel
# ===============================
from pyngrok import ngrok, conf
conf.get_default().auth_token = "YOUR_NGROK_TOKEN"

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

# ===============================
# 9️⃣ Check logs (optional)
# ===============================
!sleep 3 && tail -n 20 flask.log

🌍 Public URL: NgrokTunnel: "https://f3231bd02f79.ngrok-free.app" -> "http://localhost:8000"


📘 1️⃣4️⃣ View Logs (Optional Debugging)

Check last 20 lines:

!tail -n 20 flask.log

Check last 50 lines:

!tail -n 50 flask.log


In [ ]:
!tail -n 50 flask.log

In [ ]:
📘 1️⃣5️⃣ Project Ready 🎉

Your AI Medical Report Simplifier is fully operational.

Users can:

✔ Upload medical reports  
✔ Extract text (OCR + PDF)  
✔ Automatically simplify medical terminology  
✔ Receive explanation of abnormal values  
✔ View medical insights  
✔ Understand their report more clearly  

This tool assists patients in interpreting blood tests—
but always reminds them to consult a doctor.


📘 1️⃣6️⃣ Useful References

Tesseract OCR:
https://github.com/tesseract-ocr/tesseract

pdfplumber:
https://github.com/jsvine/pdfplumber

OPT Model:
https://huggingface.co/facebook/opt-350m

Flask Docs:
https://flask.palletsprojects.com/

pyngrok Docs:
https://pyngrok.readthedocs.io/en/latest/
